<a href="https://colab.research.google.com/github/GabKout/NLP_intent_classification/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/GabKout/NLP_intent_classification

Cloning into 'NLP_intent_classification'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 27 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (27/27), 5.55 MiB | 4.95 MiB/s, done.


In [9]:
import pandas as pd
import re
import zipfile
import json

In [ ]:
with open("/content/unzip_files/switchboard-corpus/utterances.jsonl") as json_file:
    json_list = list(json_file)

id_set = set()

data = []

for json_str in json_list:
    result = json.loads(json_str)
    data.append(result)
data[0:5]

In [56]:
dataframe = pd.DataFrame(data)

# Let's get only penultimate lines of each conversation to see if their fillers help us find their Dialog Act
dataframe["rank_in_group"] = dataframe["id"].apply(lambda x: int(re.search("-(.*)", x).group(1)))
dataframe["number_of_utt_in_group"] = dataframe.groupby("conversation_id")["rank_in_group"].transform(max)
penult_df = dataframe[dataframe["rank_in_group"] == dataframe["number_of_utt_in_group"] - 1]
penult_df["DA"] = penult_df["meta"].apply(lambda x: x["tag"][0][1])
penult_df

<ipython-input-56-f034d3a468c1>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  penult_df["DA"] = penult_df["meta"].apply(lambda x: x["tag"][0][1])


,id,conversation_id,text,speaker,meta,reply-to,timestamp,rank_in_group,number_of_utt_in_group,DA
85,4325-85,4325-0,{F Um. } /,1632,"{'tag': [['{F Um. } /', '%']]}",4325-84,None,85,86,%
120,4330-33,4330-0,-- they made me sit and wait for forty-five mi...,1635,{'tag': [['-- they made me sit and wait for fo...,4330-32,None,33,34,+
172,4103-50,4103-0,"-- {F uh, } I guess we will have to factor tha...",1564,"{'tag': [['-- {F uh, } I guess we will have to...",4103-49,None,50,51,sd
217,4327-43,4327-0,"Yeah, /{D well, } that's another problem. /I...",1619,"{'tag': [['Yeah, /', 'b'], ['{D well, } that'...",4327-42,None,43,44,b
269,4646-50,4646-0,Right. /,1670,"{'tag': [['Right. /', 'b']]}",4646-49,None,50,51,b
...,...,...,...,...,...,...,...,...,...,...
122390,4179-105,4179-0,"{D Well, } {F uh, } [ it's, + we ] just moved ...",1574,"{'tag': [['{D Well, } {F uh, } [ it's, + we ] ...",4179-104,None,105,106,sd
122477,4384-85,4384-0,Yeah. /,1653,"{'tag': [['Yeah. /', 'b']]}",4384-84,None,85,86,b
122533,4622-54,4622-0,"Oh, well that's nice. /That's good for the cl...",1528,"{'tag': [['Oh, well that's nice. /', 'ba'], [...",4622-53,None,54,55,ba
122618,4643-83,4643-0,#{D Now }# did you cut a quarter wedge in this? /,1680,{'tag': [['#{D Now }# did you cut a quarter we...,4643-82,None,83,84,qy


In [57]:
penult_df["DA"].value_counts()

fc        249
b         212
sd        186
+         144
sv         84
aa         60
%          47
ba         25
ny         24
x          24
fc^m       11
qy         10
ft          9
bd          6
nn          5
na          5
sd^t        4
bk          4
b^r         3
qh          3
o           3
ng          3
h           2
br          2
sv(^q)      2
bh          2
qy^t        2
no          2
qy^g        2
qw          2
bf          2
b^m         2
sd(^q)      2
fx          1
qy^d        1
qw^t        1
ad          1
nn^r        1
fe          1
ar          1
nd          1
fc^t        1
fw          1
ft^m        1
^2          1
Name: DA, dtype: int64